In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
%config InlineBackend.figure_format='retina'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
pd.Timestamp.now()

Timestamp('2020-05-04 16:51:17.352932')

##Import ข้อมูล COVID-19 time series

In [ ]:
'''
นำเข้่าข้อมูลจาก ศูนย์ศาสตร์แห่งระบบวิศวกรรม CSSE มหาวิทยาลัยจอห์น ฮอปกินส์ John Hopkins University
แหล่งข้อมูล: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_seriesง
โดยข้อมูลจะเป็นแบบ Time series บอกจำนวน ผู้ติดเชื้อ ผู้รักษาหายแล้วและผู้เสียชีวิต'''

#นำเข้าข้อมูลผู้ติดเชื้อ
confirmed_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
confirmed_df = pd.read_csv(confirmed_csv)

#นำเข้าข้อมูลผู้รักษาหาย
recovered_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv'
recovered_df = pd.read_csv(recovered_csv)

#นำเข้าข้อมูลผู้เสียชีวิต
deaths_csv = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
deaths_df = pd.read_csv(deaths_csv)

#รวม Data frame เข้าด้วยกัน
merge = [confirmed_df, recovered_df, deaths_df]
covid_df = pd.concat(merge, keys = ['confirmed', 'recovered', 'deaths'])

covid_df

Province/State         Country/Region  ...  5/2/20  5/3/20
confirmed 0              NaN            Afghanistan  ...    2469    2704
          1              NaN                Albania  ...     789     795
          2              NaN                Algeria  ...    4295    4474
          3              NaN                Andorra  ...     747     748
          4              NaN                 Angola  ...      35      35
...                      ...                    ...  ...     ...     ...
deaths    261            NaN         Western Sahara  ...       0       0
          262            NaN  Sao Tome and Principe  ...       1       1
          263            NaN                  Yemen  ...       2       2
          264            NaN                Comoros  ...       0       0
          265            NaN             Tajikistan  ...       2       2

[784 rows x 107 columns]

In [ ]:
#จัด Row Column ใหม่ให้อ่านง่าย จัดการง่าย
covid_df = pd.melt(covid_df, id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'])
covid_df 

,Province/State,Country/Region,Lat,Long,variable,value
0,NaN,Afghanistan,33.000000,65.000000,variable,variable
1,NaN,Albania,41.153300,20.168300,variable,variable
2,NaN,Algeria,28.033900,1.659600,variable,variable
3,NaN,Andorra,42.506300,1.521800,variable,variable
4,NaN,Angola,-11.202700,17.873900,variable,variable
...,...,...,...,...,...,...
646011,NaN,Western Sahara,24.215500,-12.885800,value,0
646012,NaN,Sao Tome and Principe,0.186360,6.613081,value,1
646013,NaN,Yemen,15.552727,48.516388,value,2
646014,NaN,Comoros,-11.645500,43.333300,value,0


In [ ]:
def read_covid_time_series():
    '''
    นำเข้่าข้อมูลจาก ศูนย์ศาสตร์แห่งระบบวิศวกรรม CSSE มหาวิทยาลัยจอห์น ฮอปกินส์ John Hopkins University
    แหล่งข้อมูล: https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
    '''
    path = 'https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_'
    status = ['Confirmed', 'Deaths', 'Recovered']
    status_dataframe = [pd.read_csv(f'{path}{s.lower()}_global.csv') for s in status]
    df=pd.concat(dfs, keys=status)
    
    dtmps=[pd.melt(df.loc[k], 
                id_vars=['Province/State', 'Country/Region', 'Lat', 'Long'],
                var_name='Date', value_name=k) for k in status]
    dx=[d.set_index(['Province/State', 'Country/Region', 'Date']) for d in dtmps]
    dcovid=pd.concat(dx, axis=1)
    dcovid.reset_index(inplace=True)
    dcovid=dcovid.loc[:, ~dcovid.columns.duplicated()]
    dcovid['Date']=pd.to_datetime(dcovid['Date'])
    dcovid=dcovid.sort_values(['Country/Region', 'Province/State', 'Date']).reset_index(drop=True)
    dcountry=dcovid.groupby(['Country/Region', 'Date']).sum()[['Confirmed', 'Deaths', 'Recovered']]
    dcountry.reset_index(inplace=True)
    dcountry['daily_new_patient']=dcountry.groupby(['Country/Region'])['Confirmed'].diff()
    dcountry['daily_new_patient_pct']=dcountry.groupby(['Country/Region'])['Confirmed'].pct_change()
    return dcountry

In [ ]:
dcountry=read_covid_time_series()
# dcountry[dcountry['Country/Region']=='Thailand']
dcountry[dcountry['Country/Region'].isin(['Thailand', 'Malaysia'])]


NameError: ignored

## Interactive chart

### normal scale

In [ ]:
dq=dcountry[['Country/Region', 'Confirmed']].groupby('Country/Region').max() > 100
country_list=dq.index[dq.Confirmed]
# country_list

@interact(country=country_list, country2=country_list)
def plot_chart(country, country2):
    df=dcountry[dcountry['Country/Region'].isin([country, country2])]
    plt.figure(figsize=(10, 6))
    sns.lineplot(data=df, x='Date', y='Confirmed', hue='Country/Region')
    plt.grid()

### normal vs log

In [ ]:
# country_list=pd.Series(dcountry['Country/Region'].unique()).sort_values() # all countries

dq=dcountry[['Country/Region', 'Confirmed']].groupby('Country/Region').max() > 100
country_list=dq.index[dq.Confirmed]
# country_list
@interact(country=country_list)
def plot_chart(country):
    df=dcountry[dcountry['Country/Region'].isin([country])]
    fig, ax = plt.subplots(figsize=(10, 6))
    sns.lineplot(data=df, x='Date', y='Confirmed', ax=ax, color='lightblue', label='normal')
    ax2=ax.twinx()
    ax2.set_yscale('log', basey=10)
    sns.lineplot(x=df.Date, y=df['Confirmed'], ax=ax2, color='salmon', label='log')
    ax2.grid()
    plt.title(f'COVID-19 in {country}')
    fig.legend()

### log vs log

In [ ]:
dq=dcountry[['Country/Region', 'Confirmed']].groupby('Country/Region').max() > 100
country_list=dq.index[dq.Confirmed]
# country_list
@interact(country=country_list, country2=country_list)
def plot_chart(country, country2):
    df=dcountry[dcountry['Country/Region'].isin([country, country2])]
    fig, ax = plt.subplots(figsize=(10, 6))
    ax.set_yscale('log', basey=10)
    ax.grid()
    sns.lineplot(data=df, x='Date', y='Confirmed', hue='Country/Region', ax=ax)

### daily new confirmed cases

In [ ]:
dq=dcountry[['Country/Region', 'Confirmed']].groupby('Country/Region').max() > 100
country_list=dq.index[dq.Confirmed]
# country_list

@interact(country=country_list, country2=country_list)
def plot_chart(country, country2):
    df=dcountry[dcountry['Country/Region'].isin([country, country2])]
    fig, ax = plt.subplots(figsize=(10, 7))
#     ax.set_yscale('log', basey=10)
    ax.grid()
    sns.lineplot(data=df, x='Date', y='daily_new_patient', hue='Country/Region', ax=ax)

## เมื่อในประเทศมีผู้ติดเชื้อเกิน 100 ราย

*   List item







In [ ]:
dcountry[dcountry.Confirmed >= 100]

In [ ]:
def confirmed_cases_gt(n=100):
    '''
    ดึงแถวที่มีค่า Confirmed มากกว่าเท่ากับค่า n ที่กำหนด เช่น 100 ราย
    '''
    dcountry=read_covid_time_series()
    df=dcountry[dcountry.Confirmed >= n].reset_index(drop=True)
    df['t'] = df.groupby('Country/Region')['Confirmed'].rank(ascending=True, method='first')
    return df

In [ ]:
dt=confirmed_cases_gt(100)
dt

In [ ]:
dq=dt[['Country/Region', 'Confirmed']].groupby('Country/Region').max() >= 100
country_list=dq.index[dq.Confirmed]
country_list

### เปรียบเทียบ 2 ประเทศ

In [ ]:
dq=dt[['Country/Region', 'Confirmed']].groupby('Country/Region').max() >= 100
country_list=dq.index[dq.Confirmed]
# country_list
@interact(country=country_list, country2=country_list, yscale=['log', 'normal'])
def plot_chart(country, country2, yscale):
    sns.set(style="ticks", rc={"lines.linewidth": 0.7})
    df=dt[dt['Country/Region'].isin([country, country2])]
    fig, ax = plt.subplots(figsize=(16, 6))
    if yscale=='log':
        ax.set_yscale('log', basey=10)
    ax.grid()
    sns.pointplot(data=df, x='t', y='Confirmed', hue='Country/Region', ax=ax)
#     sns.lineplot(data=df, x='t', y='Confirmed', hue='Couccntry/Region', ax=ax)
    plt.title(f'COVID-19 after 100 confirmed cases (as {pd.Timestamp.now()})')
    # save chart to file
    plt.savefig(f'100days_{country}_{country2}.svg')

### แสดงหลาย ๆ ประเทศพร้อม ๆ กัน

In [ ]:
def plot_chart(countries, yscale='log'):
    sns.set(style="ticks", rc={"lines.linewidth": 1})
    df=dt[dt['Country/Region'].isin(countries)]
    fig, ax = plt.subplots(figsize=(16, 9))
    if yscale=='log':
        ax.set_yscale('log', basey=10)
    ax.grid()
    sns.pointplot(data=df, x='t', y='Confirmed', hue='Country/Region', ax=ax)
#     sns.lineplot(data=df, x='t', y='Confirmed', hue='Country/Region', ax=ax)
    plt.title(f'COVID-19 after 100 confirmed cases (as {pd.Timestamp.now()})')

countries=['Thailand', 'Italy', 'United Kingdom', 'US']
plot_chart(countries, 'log')

# save chart to file
# plt.savefig('100days.png')
plt.savefig('100days.svg')

NameError: ignored